# Introduction to Object Recognition

## Overview

In other lessons we have primarily dealt with classification problems. We have worked with images with a single object which we classify into its category.    

**Object recognition** is a more difficult task. The process allows us to deal with complex images containing multiple objects. There are two major sets in this process:    
1. **Detect** the presence of each of multiple objects in an image. An added difficulty is the trade-off between correctly detecting the objects of interest and falsely detecting spurious objects in the background.  
2. Compute a minimum size **bounding box** around each object. 
3. The object detected in each of the bounding box is **classified**. 

Our goal is to give you an understanding of how modern object recognition algorithms work. Given the research momentum in this area it is entirely likely that there will be better performing algorithms by the time you read this. 

The object recognition problem has been around for quite some time. Historically, object recognition was performed using hand engineered features. Only since 2013 there has been a surge of both interest and performance of object recognition with the introduction of deep learning methods. These algorithms no longer require the cumbersome process of hand engineering features. Some key papers in the history of deep object recognition are:

1. In 2013 [Erhan et. al.](https://arxiv.org/pdf/1312.2249.pdf) published Scalable Object Detection using Deep Neural Networks, which introduced the R-CNN algorithm the first widely accepted deep learning object detection algorithm. R-CNN demonstrated a significant improvement in object recognition accuracy. However, this algorithm proved to be too slow for real-time video processing. 
2. The Fast R-CNN algorithm was introduced by [Girshick](https://arxiv.org/pdf/1504.08083.pdf) Fast R-CNN simplified the required computations but still struggled with real-time video.  
3. Further improvements by [Ren et. al.](https://arxiv.org/pdf/1506.01497.pdf) lead to the Faster R-CNN algorithm. However, the computational complexity of this algorithm was still rather high.  

All of these algorithms share a similar architecture. This architecture is in the form of a pipeline with process steps computed sequentially. The key steps in these pipelines generally include: 

1. **Convolutional Neural Network:** The CNN creates features which which are used to detect and then classify objects in the image.   
2. **Candidate bounding boxes:** Candidate bounding boxes are generated. Multiple candidate bounding boxes cover each region. 
3. **Filtering of bounding boxes:** The probability of an object being in each bounding box is computed and low probability boxes are filtered. 
4. **Minimal bounding boxes:** The size of the bounding boxes must be adjusted to best fit the objects detected. 
5. **Classification:** The objects in the bounding box are classified. 

The result is that training and performing inference with these complex pipelines has significant computational complexity. Further, these pipelines can be difficult to train.

## The YOLO Algorithms

In this lesson we will not focus on the R-CNN family of algorithms. Instead, we will focus on the You Only Look Once or YOLO algorithms. YOLO was first introduced in by [Redmon et. al.](https://arxiv.org/pdf/1506.02640v5.pdf) in 2016. 

The YOLO algorithms exhibit greater accuracy than R-CNN and its derivatives. In most cases, YOLO is considerably more efficient and can truly be applied to real-time video. At the time of this writing the YOLO family of algorithms are generally considered to be the state of the art in object recognition. YOLO does have some limitations. For example, it is known that YOLO algorithms do not perform well when faced with a large number of small objects, such as a flock of birds.      

The breakthrough with YOLO is the recognition that object detection and classification can be performed in one pass. Hence the name You Only Look Once. All of the R-CNN algorithms required multiple passes to perform object recognition, which was computationally intensive. 

Since the original YOLO paper there have been two improved versions of the YOLO algorithms have been published. The latest information on YOLO can be found on the [YOLO web pages](https://pjreddie.com/darknet/yolo/), which is part of the Darknet repository. The [YOLO9000 algorithm](https://arxiv.org/pdf/1612.08242.pdf), which is often referred to as YOLO v2, is a significant improvement over the original YOLO algorithm. Lately, some incremental improvements have been published as [YOLO v3](https://pjreddie.com/media/files/papers/YOLOv3.pdf).

Unlike previous object detection algorithms YOLO uses regression methods rather than classification methods. This approach is key to the efficiency of YOLO. All of the model parameters are computed in one regression step.  

### IOU

Object detection requires a metric to determine how well the computed bounding box fits the object. The metric we use is known as Intersection over Union or IoU. This metrics is used to compare human marked bounding boxes with the ones computed by the algorithm. 

The concept of IoU is illustrated in the figure below:

<img src="img/IoU.jpg" alt="Drawing" style="width:600px; height:300px"/>
<center>**Intersection and union of bounding boxes**</center>

Given the two bounding boxes the IoU is easily computed as:

$$IoU = \frac{Area\ of\ intersection}{Area\ of\ union}$$

### Bounding box parameterization

How do we perform object detection and compute a bounding box using a regression model? First, we need a parameterization for the bounding box suitable for regression. There are 4 spacial parameters along with the probability that an object is in the box. For each bounding box the parameterization is:   

$$
b_x = \sigma(t_x) + c_x\\
b_y = \sigma(t_y) + c_y\\
b_w = p_w e^{t_w}\\
b_h = p_h e^{t_h}\\
p_0 = Pr(object) * IoU(b,\ object) = \sigma(t_0)
$$

These relationships are illustrated in the figure below. The 

<img src="img/BoundingBox1.jpg" alt="Drawing" style="width:600px; height:300px"/>
<center>**Proposal bounding box in blue and optimized bounding box in red**</center>

The above relationships include priors, often known as proposals, for the parameters of the bounding box. These are the initial values of for the regression and are illustrated in the figure above in blue.      
$c_x$ is the prior of the x coordinate of the bounding box,   
$c_y$ is the prior of the y coordinate of the bounding box,     
$b_w$ is the prior of the width of the bounding box, and     
$b_h$ is the prior of the height of the bounding box.    

In the above figure the following notation is used to parameterize the best fit bounding box, which is shown in red: 
$b_x$ is the x coordinate of the center of the bounding box,   
$b_y$ is the y coordinate of the center of the bounding box,  
$b_w$ is the width of the bounding box,   
$b_h$ is the height of the bounding box, and   
$p_0$ is the probability of an object in the box.    


### Object detection by regression

Now that we have a parameterization of the object detection problem let's formulate the regression model. Using this parameterization, the best fit bounding box is computed using a regression model. Additionally, we need to classify the object in the bounding box. Our goal is to compute the smallest bounding box that encloses an object along with the probability there is an object in the box and the class of the object. By using a regression model to estimate all of the parameters 

Training the regression model uses labeled or known cases. The known cases typically have human marked bounding boxes around known objects. The presence and class of the object are also labeled.  

To illustrate the concepts we will start with just a single bounding box with three possible object classes. The vector of parameters to be estimated in this regression problem can be written as follows:

$$
\boldsymbol{\hat{y}} = 
\begin{bmatrix}b_x \\ b_y \\ b_w \\ b_h \\ p_0 \\ c_1 \\ c_2 \\ c_3 \end{bmatrix} 
$$

The first 5 parameters in this vector have already been discussed. The values of $\{ c_1, c_2, c_3 \}$ are binary, and indicate which class the object falls in. Typically, the regression model is trained by minimizing the RMS error with the labeled values of the parameters. 

Next, let's expand the simple single bounding box model to a multiple bounding boxes. An example of multiple bounding box proposals sharing a single center is shown in the figure below. An object could have various shapes, resulting in a higher probability for one of the bounding boxes enclosing the object than the others.     

<img src="img/MultiBox.jpg" alt="Drawing" style="width:300px; height:300px"/>
<center>**Bounding boxes with common center or anchor point**</center>

The above figure shows multiple bounding boxes with a common center. On a real image there will be many possible centers each with multiple bounding box proposals. We can generalize the vector of model parameters of the regression model as shown here. The parameters are repeated in the vector for each bounding box.     

$$
\boldsymbol{\hat{y}} = 
\begin{bmatrix}b_x \\ b_y \\ b_w \\ b_h \\ p_0 \\ c_1 \\ c_2 \\ c_3 \\ \vdots \\
b_x \\ b_y \\ b_w \\ b_h \\ p_0 \\ c_1 \\ c_2 \\ c_3 \end{bmatrix} 
$$



### Multi-scale feature generation with CNNs

Good object recognition requires good feature generation. As has already been mentioned, the application of recurrent neural networks for feature generation in object recognition was a major breakthrough.   

A recent advance in object recognition is the application of multi-scale features using the hierarchy of RNNs. This method was first introduced by [Lin et all](https://arxiv.org/pdf/1612.03144.pdf) in 2017. 

We have discussed RNNs in another lesson. In that lesson we saw how the scale of the features changes  The heirarchy of RNN layers creates features features 

### Non-max Suppression

The result of the regression produces the optimal dimensions of the bounding boxes, the probabilities and the classification of the object. There can be multiple bounding boxes covering the same area of the image. 

Packages to install
- h5py
- pillow
- pydot
- graphviz

Download the yolo.h5 weights from

https://github.com/OlafenwaMoses/ImageAI/releases/download/1.0/yolo.h5   

Note that this file is 237 MB. You will need to have adequate disk space. 

### Intersection Over Union 

